<a href="https://colab.research.google.com/github/Federico6419/MachineLearningProject/blob/main/MachineLearningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraries


In [1]:
!pip install gymnasium
!pip install swig     #This solves the errori in the installation of gymnasium[box2d]
!pip install gymnasium[box2d]
!pip install gym-notebook-wrapper   #This installs Gym-Notebook-Wrapper, that provides small wrappers for running and rendering OpenAI Gym

#To solve the xvfb missing file problem
!sudo apt-get install xvfb
!pip install xvfbwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2373076 sha256=f98400cec2eedf0fd35c78052c5edebb1ce17d25e40ae52276696685b9fa6118
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkb

## Import libraries

In [2]:
!git clone https://github.com/Federico6419/MachineLearningProject          #It clones my github repository
%cd MachineLearningProject

import gymnasium as gym
import gnwrapper
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

import cv2

import config
from model import Model

from collections import deque

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Cloning into 'MachineLearningProject'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 145 (delta 8), reused 4 (delta 4), pack-reused 130
Receiving objects: 100% (145/145), 93.73 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/MachineLearningProject


# F

In [69]:
episode_reward = 0
#Add negative reward?
buffer = deque([], config.BUFFER_SIZE)             #Initialize the Queue that contains the past experience
epsilon = config.MAX_EPSILON
alpha = config.ALPHA
decay = config.EPSILON_DECAY

#For the plotting
cum_reward_table = np.zeros(config.NUM_EPISODES)
cum_reward_nn = np.zeros(config.NUM_EPISODES)

#Initialize the Model
model = Model().to(config.DEVICE)

#Initialize the Target Model
target_model = Model().to(config.DEVICE)

optimizer = optim.Adam(model.parameters(), lr=config.LR)
optimizer_target = optim.Adam(target_model.parameters(), lr=config.LR)

huber_loss=nn.HuberLoss(delta=1.0)

action_space = [
                (-1, 1, 0.2), (0, 1, 0.2), (1, 1, 0.2),
                (-1, 1,   0), (0, 1,   0), (1, 1,   0),               #(Steering Wheel, Gas, Break)
                (-1, 0, 0.2), (0, 0, 0.2), (1, 0, 0.2),               #Range -1~1 0~1 0~1
                (-1, 0,   0), (0, 0,   0), (1, 0,   0)
              ]

#Define the policy to know how chose the action
#Q-Table
def select_action(state, epsilon):
    rv = random.uniform(0, 1)
    if rv < epsilon:
        return env.action_space.sample()
    else:
        return np.argmax(Q[state])

#Neural Network
def select_action_nn(state, epsilon):
    rv = random.uniform(0, 1)
    if rv < epsilon:
        return env.action_space.sample(), random.randrange(len(action_space))          #We sample a random action

    else:
        prediction = model(state.type('torch.cuda.FloatTensor')).detach().cpu().numpy()
        action=np.argmax(prediction)              #Select the action with the maximum predicted Q-Value

        return prediction, action


## update the epsilon value along the iteration until converges to MIN_EPSILON
def update_epsilon(epsilon):
    epsilon -= epsilon/100 # reduce epsilon by 1/100
    if epsilon<=config.MIN_EPSILON:
        return config.MIN_EPSILON
    else:
        return epsilon

## update the epsilon every episode by epsilon decay variable
def update_epsilon_nn(epsilon):
    epsilon *= decay
    if epsilon<=config.MIN_EPSILON:
        return config.MIN_EPSILON
    else:
        return epsilon


env = gym.make("CarRacing-v2", render_mode="human")


if(config.USE_QTABLE):
    # define the Q table
    #Q = np.zeros([27684, env.action_space.n]) # little discretization
    Q = np.zeros([19051200, env.action_space.n]) #big discretization

###see the limit of the values of the box observation space
#print(env.observation_space.high)
#print(env.observation_space.low)

###see in more detail the action space and the observation space
#print(env.action_space)
#print(env.observation_space)


if(config.USE_QTABLE): # use a q table to reach the goal
    for i in range(config.NUM_EPISODES):
        observation, info = env.reset()# use seed to have same initial state
        #state = config.discretize(observation)
        state = config.big_discretize(observation)

        for j in range(500):
            action = select_action(state,epsilon)
            obv, reward, done, truncated, info = env.step(action)
            #next_state = config.discretize(obv)
            next_state = config.big_discretize(obv)

            next_max = np.max(Q[next_state])

            Q[state,action] += alpha*(reward+config.GAMMA*next_max-Q[state,action])
            state = next_state

            episode_reward += reward

            if done or truncated:
                break

        print("episode: ", i)
        print("episode cumulative reward : ", episode_reward)
        print("epsilon: ",epsilon)
        epsilon = update_epsilon(epsilon)
        cum_reward_table[i]=episode_reward
        episode_reward = 0 #reset the total reward each episode

    #save the q table for testing
    #np.savetxt('q_table.csv', Q, delimiter=','fmt='%f18')
    #np.savetxt('q_table_little_discretization2000.csv', Q, delimiter=',') # full precision
    np.savetxt('q_table_big_discretization1000.csv', Q, delimiter=',') # full precision

else:             #Use a Neural Network to approximate the Q Function
    for i in range(config.NUM_EPISODES):
        state, info = env.reset()               #The state is a 96x96 Matrix, that contains elements composed by 3 Colours RGB
        state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)           #Convert the state into a Grayscale Image, that is a Matrix 96x96 composed by Integer values
        #Add normalization?

        frames_queue = deque([state]*3, maxlen = 3)

        for j in range(500):
            #ATTENZIONE, NON SO SE CONVERTIRE IN NP ARRAY E POI IN TENSOR SIA GIUSTO
            current_frame = np.array(frames_queue)
            current_frame = torch.from_numpy(current_frame)
            #current_frame = np.transpose(current_frame, (1, 2, 0))

            #action = select_action_nn(state, epsilon)
            action, action_id = select_action_nn(current_frame, epsilon)                      #The Action is composed by 3 Values, that are the steering, gas and breaking
            next_state, reward, done, truncated, info = env.step(action)

            episode_reward += reward

            next_state = cv2.cvtColor(next_state, cv2.COLOR_BGR2GRAY)
            #Add normalization?

            #ATTENZIONE, NON SO SE CONVERTIRE IN NP ARRAY E POI IN TENSOR SIA GIUSTO
            frames_queue.append(next_state)
            next_frame = np.array(frames_queue)
            next_frame = torch.from_numpy(next_frame)
            #next_frame = np.transpose(next_frame, (1, 2, 0))

            #Remove the oldest item if the queue is full, in a way such that we can add a new one
            if len(buffer)>=config.BUFFER_SIZE:
                buffer.popleft()               #We dequeue the oldest item

            #buffer.append([*state,action,reward,*next_state,done])
            #buffer.append((state, action, reward, next_state, done))
            buffer.append((current_frame, action_id, reward, next_frame, done))

            #next_state = cv2.cvtColor(next_state, cv2.COLOR_BGR2GRAY)
            #state = next_state    #We update the Current State

            #Let's train the Neural Network every 4 actions and if the buffer has at least BATCH_SIZE elements
            if((len(buffer) >= config.BATCH_SIZE) and ((j+1) % 4 == 0)):
                batch = random.sample(buffer, config.BATCH_SIZE)
                dataset = np.array(batch)

                array_states = []
                array_targets = []
                array_actions = []

                #print(len(batch[0][0]))

                #states = torch.from_numpy((dataset[:,:8]).astype('float32'))
                #actions = torch.from_numpy(dataset[:,8:9].astype('int64'))
                #rewards = torch.from_numpy(dataset[:,9:10].astype('float32'))
                #next_states = torch.from_numpy((dataset[:,10:18]).astype('float32'))
                #dones = torch.from_numpy(dataset[:,18:19].astype('float32'))

                for current_frame, action, reward, next_frame, done in dataset:
                    #states = torch.from_numpy((dataset[:,:0]).astype('uint8'))
                    #states = dataset[0]
                    #actions = torch.from_numpy(dataset[:,1:1].astype('float32'))
                    #rewards = torch.from_numpy(dataset[:,2].astype('float32'))
                    #next_states = torch.from_numpy((dataset[:,3:3]).astype('uint8'))
                    #next_states = dataset[0][3]
                    #dones = torch.from_numpy(dataset[:,4].astype('float32'))

                    #-------vanilla dqn------------#

                    """# Find next best action so can compute the next reward for the target
                    #predictions_next = target_model(next_states).detach().max(1)[0].unsqueeze(1)
                    #next_actions=np.argmax(predictions_next) # Select action with max Q-value

                    #Compute corresponding (predicted) reward of next state
                    #next_rewards = predictions_next[next_actions]
                    next_rewards = target_model(next_states).detach().max(1)[0].unsqueeze(1)
                    #-------------------------------#"""
                    #---------double dqn-------------#

                    # Find next best action using model network
                    #predictions_next = model(next_frame.type('torch.cuda.FloatTensor')).detach().numpy()
                    predictions_next = model(next_frame.type('torch.cuda.FloatTensor')).detach().cpu().numpy()
                    #predictions_next = model(np.expand_dims(next_states, axis=0)).detach().numpy()
                    #next_actions.append(predictions_next)

                    ########################
                    t = target_model(next_frame.type('torch.cuda.FloatTensor'))
                    predictions_next[action] = reward + config.GAMMA * max(t)

                    #array_states.append(state)
                    #array_targets.append(predictions_next)
                    #array_actions.append(action)
                    ########################


                    #next_actions = np.argmax(predictions_next,axis=1) # Select action with max Q-value
                    #array_states = np.argmax(next_actions) # Select action with max Q-value
                    #array_states =  array_states[..., np.newaxis]


                    #evaluate Q(s',a') founded by model using the target network
                    #next_rewards = target_model(next_frame.type('torch.cuda.FloatTensor')).gather(1, torch.from_numpy(next_actions))
                    #next_rewards = torch.from_numpy(evaluations[next_actions])

                    #-------------------------------#

                    #targets = rewards + config.GAMMA_NN*next_rewards*(1-dones)
                    #targets = reward + config.GAMMA_NN*next_rewards*(1-done)

                    #compute the predicted value of the model(output)
                    output = model(current_frame.type('torch.cuda.FloatTensor'))#.gather(1, array_actions)
                    #compute the huber loss
                    output =  output[..., np.newaxis]
                    predictions_next =  predictions_next[..., np.newaxis]
                    predictions_next = torch.from_numpy(predictions_next).to(config.DEVICE)
                    loss = huber_loss(output, predictions_next)

                    #Train network
                    optimizer.zero_grad()#clear existing gradient
                    loss.backward() #backpropagate the error
                    optimizer.step() # update weights

        if (i+1) % 25 == 0:
            #save the weight of the network
            config.save_model(model,optimizer,i+1)
            print("Save weigths in: "+ config.CHECKPOINT_FOLDER)

        if (i+1) % 5 == 0:
            #update weights of target network every 10 actions
            #if len(buffer) >= config.BATCH_SIZE and (j+1) % config.TARGET_FREQ_UPDATE == 0:
            if (j+1) % config.TARGET_FREQ_UPDATE == 0:
                print("Target network updated")
                config.load_model(config.CHECKPOINT_FOLDER,target_model,optimizer_target)

        if done or truncated:
            epsilon = update_epsilon_nn(epsilon)
            print("episode ", i)
            print("episode cumulative reward: ", episode_reward)
            print("current epsilon: ", epsilon)
            print("#---------------------------------------------#")
            break

        cum_reward_nn[i]=episode_reward
        episode_reward = 0


env.close()

<ipython-input-69-2a16e6124d92>:163: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  dataset = np.array(batch)
<ipython-input-69-2a16e6124d92>:163: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dataset = np.array(batch)


KeyboardInterrupt: ignored

## Example

In [ ]:
env = gnwrapper.Animation(CarRacing())
env = CarRacing(render_mode="rgb_array")
#env = CarRacing(render_mode="state_pixels")
#env = gym.make("CarRacing-v2", domain_randomize=True, render_mode="state_pixels")

env.reset()
#env.render()
#im = env.render()
#im = env.render("state_pixels")

#plt.imshow(im)

for i in range(40):
  env.step(action=[-0.3,1,0])
  im = env.render()
  plt.imshow(im)

"""
def state_image_preprocess(state_image):
    state_image = state_image.transpose((2,0,1))
    state_image = np.ascontiguousarray(state_image, dtype=np.float32) / 255
    state_image = torch.from_numpy(state_image)
    return state_image.unsqueeze(0).to(device)

state_image_preprocess(im).shape
plt.imshow(state_image_preprocess(im).cpu().squeeze(0).permute(1, 2, 0).numpy())
"""

NameError: ignored